# TIA125 - Assignment

## Description

This part contains explonations regarding the columns in the data set
```
cohort
degree-seeking cohort type 
    ('4y bach' = Bachelor's/equivalent-seeking cohort at 4-year institutions; 
    '4y other' = Students seeking another type of degree or certificate at a 4-year institution; 
    '2y all' = Degree-seeking students at 2-year institutions)

race/ethnicity of students 
    ('X' = all students; 
    'Ai' = American Indian; 
    'A' = Asian; 
    'B' = Black; 
    'H' = Hispanic; 
    'W' = White)

grad_100: finnishing in timeframe of the programme
grad_100_rate: same as above in procent

grad_150: 1.5x time study speed
grad_150_rate: same as above in procent

grad_cohort: 
    Number of first-time, 
    full-time, 
    degree-seeking students in the cohort being tracked, 
    minus any exclusions
```

## Data processing

### Libraries

In [4]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt

import sys
sys.path.append("../src/utility/")
from utility import plot_settings
from utility import visualization
from utility import data_base as data

### Data cleaning

In [5]:
df_raw = data.read_csv('../data/cc_institution_grads.csv')
print(f'Raw data file: {len(df_raw)}')
df_drop_na = df_raw.dropna()
print(f"rows after droppped Nan: {len(df_drop_na)}")
df = df_drop_na.copy()

Raw data file: 1302102
rows after droppped Nan: 332061


In [6]:
df_raw.head()

,unitid,year,gender,race,cohort,grad_cohort,grad_100,grad_150,grad_100_rate,grad_150_rate
0,100760,2011,B,X,2y all,446.0,73.0,105.0,16.4,23.5
1,100760,2011,M,X,2y all,185.0,NaN,40.0,NaN,21.6
2,100760,2011,F,X,2y all,261.0,NaN,65.0,NaN,24.9
3,100760,2011,B,W,2y all,348.0,NaN,86.0,NaN,24.7
4,100760,2011,M,W,2y all,162.0,NaN,35.0,NaN,21.6


In [7]:
df.head()

,unitid,year,gender,race,cohort,grad_cohort,grad_100,grad_150,grad_100_rate,grad_150_rate
0,100760,2011,B,X,2y all,446.0,73.0,105.0,16.4,23.5
18,100760,2012,B,X,2y all,594.0,40.0,87.0,6.7,14.6
36,100760,2013,B,X,2y all,594.0,46.0,54.0,7.7,9.1
54,101028,2011,B,X,2y all,261.0,25.0,42.0,9.6,16.1
72,101028,2012,B,X,2y all,281.0,41.0,41.0,14.6,14.6


In [8]:
df_clean = df[['unitid', 'year', 'gender', 'race', 'grad_100', 'grad_150', 'grad_100_rate', 'grad_150_rate']].copy()

### Looking at the raw data and clean data (top 5 with .head())